<a href="https://colab.research.google.com/github/englissi/englissi/blob/main/%EB%85%BC%EB%AC%B8%EC%93%B0%EC%9E%90.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
!pip install gradio

In [3]:
import gradio as gr
import pandas as pd

# Sample structure of categories and phrases
categories = {
    "Establishing why your topic (X) is important": [
        ("X is the main / leading / primary / major cause of ..", "X는 주요 원인입니다.."),
        ("Xs are a common / useful / critical part of…", "Xs는 일반적이고 유용하며 중요한 부분입니다…"),
        ("Xs are among the most widely used / commonly discussed / well-known / well-documented / widespread / commonly investigated types of …", "Xs는 가장 널리 사용되고 / 자주 논의되고 / 잘 알려져 있고 / 잘 문서화되어 있고 / 널리 퍼져 있으며 / 일반적으로 조사된 유형 중 하나입니다..."),
        ("X is recognized as being / believed to be / widely considered to be the most important …", "X는 가장 중요한 것으로 인정되고 / 믿어지고 / 널리 간주됩니다..."),
        ("It is well known / generally accepted / common knowledge that X is …", "X는 잘 알려져 있고 / 일반적으로 받아들여지며 / 공통적으로 알려져 있습니다...")
    ],
    "Outlining the past-present history of the study of X": [
        ("Last century X was considered to be / viewed as / seen as the most …", "지난 세기 X는 가장 중요한 것으로 간주되었습니다..."),
        ("Initial / Preliminary / The first studies of X considered it to be", "초기 / 예비 / 첫 번째 연구는 X를 고려했습니다"),
        ("Traditionally X / In the history of X, the focus has always been …", "전통적으로 X / X의 역사에서, 초점은 항상 ...에 있었습니다"),
        ("Scientists / Researchers / Experts have always seen X as …", "과학자 / 연구자 / 전문가들은 항상 X를 ...로 보았습니다"),
        ("Until now / For many years / Since 1993 Xs have been considered as …", "지금까지 / 수년 동안 / 1993년부터 X는 ...로 간주되었습니다")
    ],
    "Outlining the possible future of X": [
        ("The next decade is likely to see / witness a considerable rise in X", "다음 10년은 X의 상당한 증가를 볼 가능성이 있습니다"),
        ("In the next few years X will become / is likely to have become", "다음 몇 년 동안 X는 ...이 될 것입니다 / 될 가능성이 있습니다"),
        ("Within the next few years, X is set / destined / likely to become an important component in …", "다음 몇 년 내에 X는 ...의 중요한 구성 요소가 될 것입니다 / 될 가능성이 있습니다"),
        ("By 2025 / Within the next ten years, X will have become …", "2025년까지 / 다음 10년 내에 X는 ...이 될 것입니다"),
        ("X will soon / shortly / rapidly / inevitably be an issue that …", "X는 곧 / 빠르게 / 필연적으로 ... 문제가 될 것입니다")
    ],
    "Indicating the gap in knowledge and possible limitations": [
        ("Few researchers have addressed the problem / issue / question of …", "소수의 연구자들만이 ... 문제를 다루었습니다"),
        ("Previous work has only focused on / been limited to / failed to address …", "이전 연구는 ...에만 집중되었고 / 제한되었고 / 다루지 못했습니다"),
        ("A basic / common / fundamental / crucial / major issue of …", "기본적인 / 일반적인 / 근본적인 / 중요한 / 주요 문제는 ...입니다"),
        ("The central / core problem of", "중심 / 핵심 문제는"),
        ("A challenging / An intriguing / An important / A neglected area in the field of …", "도전적인 / 흥미로운 / 중요한 / 방치된 분야는 ...입니다")
    ],
    "Stating the aim of your paper and its contribution": [
        ("In this report / paper / review / study we …", "이 보고서 / 논문 / 리뷰 / 연구에서 우리는 ..."),
        ("This paper outlines / proposes / describes / presents a new approach to …", "이 논문은 ...에 대한 새로운 접근 방식을 개요 / 제안 / 설명 / 제시합니다"),
        ("This paper examines / seeks to address / focuses on / discusses / investigates how to solve …", "이 논문은 ...을 해결하는 방법을 검토 / 해결하고자 함 / 집중 / 논의 / 조사합니다"),
        ("This paper is an overview of / a review of / a report on / a preliminary attempt to …", "이 논문은 ...에 대한 개요 / 리뷰 / 보고서 / 예비 시도입니다"),
        ("The present paper aims to validate / call into question / refute Peng’s findings regarding …", "현재 논문은 ...에 관한 펭의 발견을 검증 / 의문을 제기 / 반박하는 것을 목표로 합니다")
    ],
    "Explaining the key terminology in your field": [
        ("The term ‘X’ is generally understood to mean / has come to be used to refer to / has been applied to …", "용어 'X'는 일반적으로 ...을 의미하는 것으로 이해되며 / ...을 참조하는 데 사용되었으며 / ...에 적용되었습니다"),
        ("In the literature, X usually refers / often refers / tends to be used to refer to …", "문헌에서 X는 보통 ...을 의미하며 / 종종 ...을 의미하며 / ...을 의미하는 데 사용되는 경향이 있습니다"),
        ("In the field of X, several / various / many definitions of Y can be found.", "X의 분야에서는 Y의 여러 / 다양한 / 많은 정의가 발견될 수 있습니다"),
        ("The term X is / was / has been used by Molotov [2011] to refer to …", "용어 X는 ...을 참조하기 위해 Molotov [2011]에 의해 사용되었습니다"),
        ("Molotov uses the term X [2011] to refer to / denominate …", "Molotov은 ...을 참조하기 위해 / 명명하기 위해 용어 X [2011]을 사용합니다")
    ],
    "Explaining how you will use terminology and acronyms in your paper": [
        ("The acronym PC stands for / denotes …", "약어 PC는 ...을 의미합니다"),
        ("The subjects (henceforth named / hereafter ‘X’) are…", "주제 (이후 'X'라고 명명됨)는..."),
        ("The subject, which we shall call / refer to as ‘X’, is …", "우리가 'X'라고 부를 주제는 ...입니다"),
        ("Throughout this paper / section we use the terms ‘mafia’ and ‘the mob’ interchangeably, following / in accordance with the practice of this department where this study was conducted.", "이 논문 / 섹션 전체에서 우리는 '마피아'와 '갱단'이라는 용어를 이 연구가 수행된 이 부서의 관행에 따라 교체하여 사용합니다"),
        ("The fonts, i.e. / that is to say the form of the characters, are of various types.", "글꼴, 즉 문자 형식은 다양한 유형입니다")
    ],
    "Giving the structure of paper - what is and is not included": [
        ("This paper is organized as follows / divided into five sections.", "이 논문은 다음과 같이 구성되어 있습니다 / 다섯 섹션으로 나뉩니다"),
        ("The first section / Section 1 gives a brief overview of …", "첫 번째 섹션 / 섹션 1은 ...의 간략한 개요를 제공합니다"),
        ("The second section examines / analyses …", "두 번째 섹션은 ...을 검토 / 분석합니다"),
        ("In the third section a case study is presented / analyzed …", "세 번째 섹션에서는 사례 연구가 제시 / 분석됩니다"),
        ("A new methodology is described / outlined in the fourth section …", "새로운 방법론이 네 번째 섹션에서 설명 / 개요됩니다")
    ],
    "Giving general panorama of past-to-present literature": [
        ("There is a considerable / vast amount of literature on …", "...에 대한 상당한 / 방대한 문헌이 있습니다"),
        ("In the literature there are many / several / a surprising number of / few examples of …", "문헌에는 ...의 많은 / 여러 / 놀라운 수의 / 몇 가지 예가 있습니다"),
        ("What we know / is known about X is largely based on …", "우리가 알고 있는 / X에 대해 알고 있는 것은 주로 ...에 기반합니다"),
        ("Much / Not much / Very little is known about …", "...에 대해 많이 / 거의 / 매우 적게 알려져 있습니다"),
        ("Many / Few studies have been published on … [Ref]", "...에 대한 많은 / 적은 연구가 발표되었습니다 [참조]")
    ],
    "Reviewing past literature": [
        ("In their seminal / groundbreaking / cutting edge paper of 2001, Peters and Jones …", "2001년 Peters와 Jones의 중요한 / 획기적인 / 최첨단 논문에서 ..."),
        ("Initial / Preliminary work in this field focused primarily on …", "이 분야의 초기 / 예비 작업은 주로 ...에 초점을 맞추었습니다"),
        ("Some preliminary work was carried out in the early 1990s / several years ago …", "1990년대 초 / 몇 년 전 몇 가지 예비 작업이 수행되었습니다"),
        ("Doyle in 2000 was among / one of the first to …", "2000년 Doyle은 ...을 처음으로 한 사람 중 하나였습니다"),
        ("The first investigations into / studies on X found that …", "X에 대한 첫 번째 조사는 ...을 발견했습니다")
    ],
    "Reviewing subsequent and more recent literature": [
        ("Experiments on X were conducted / carried out / performed on X in 2009 by a group of researchers from …", "2009년에 ...의 연구 그룹에 의해 X에 대한 실험이 수행되었습니다"),
        ("In a major advance in 2010, Berlusconi et al. surveyed / interviewed …", "2010년에 중요한 진전으로 Berlusconi 등은 ...을 조사했습니다"),
        ("Jeffries and co-workers [2011] measured / calculated / estimated …", "Jeffries와 공동 연구자들은 [2011] 측정 / 계산 / 추정했습니다"),
        ("In [67] the authors investigated / studied / analyzed …", "[67]에서 저자들은 ...을 조사했습니다"),
        ("A recent review of the literature on this topic / subject / matter / area [2012] found that …", "이 주제 / 주제 / 문제 / 영역에 대한 최근의 문헌 검토 [2012]는 ...을 발견했습니다")
    ],
    "Reporting what specific authors have said": [
        ("In her analysis / review / overview / critique of X, Bertram [2] questions the need for …", "그녀의 X 분석 / 리뷰 / 개요 / 비판에서 Bertram [2]은 ...의 필요성을 질문합니다"),
        ("In his introduction to / seminal article on / investigation into X, Schneider [3] shows that …", "X에 대한 그의 소개 / 중요한 기사 / 조사에서 Schneider [3]은 ...을 보여줍니다"),
        ("Dee [4] developed / reported on a new method for X and concluded that …", "Dee [4]는 X에 대한 새로운 방법을 개발 / 보고하고 ...이라고 결론지었습니다"),
        ("Southern’s group [5] calls into question some past assumptions / hypotheses / theories about X.", "Southern의 그룹 [5]은 X에 대한 과거 가정 / 가설 / 이론을 의문시합니다"),
        ("Burgess [6], an authority on X, notes / mentions / highlights / states / affirms that …", "X의 권위자인 Burgess [6]은 ...을 언급 / 언급 / 강조 / 진술 / 확인합니다")
    ],
    "Mentioning positive aspects of others’ work": [
        ("Smith’s [22] use of X is fully justified / very plausible / endorsed by experience.", "Smith [22]의 X 사용은 완전히 정당화되었고 / 매우 그럴듯하고 / 경험에 의해 승인되었습니다"),
        ("Kamos’s [23] assumptions seem to be realistic / well-founded / well-grounded / plausible / reasonable / acceptable.", "Kamos [23]의 가정은 현실적이고 / 잘 기반이 잡혀 있으며 / 타당하고 / 합리적이고 / 수용 가능해 보입니다"),
        ("The equations given in [24] are accurate / comprehensive …", "[24]에 주어진 방정식은 정확하고 / 포괄적입니다"),
        ("It has been suggested [25] that … and this seems to be a reliable / useful / innovative approach …", "[25]에서 제안되었으며 ... 이것은 신뢰할 수 있고 / 유용하며 / 혁신적인 접근 방식으로 보입니다")
    ],
    "Highlighting limitations of previous studies - authors not mentioned by name": [
        ("Research has tended to focus on X rather than Y. An additional problem is that / Moreover X is …", "연구는 Y보다 X에 집중하는 경향이 있습니다. 추가 문제는 ...이며 / 또한 X는 ...입니다"),
        ("The main limitation / downside / disadvantage / pitfall / shortfall of X is …", "X의 주요 한계 / 단점 / 단점 / 함정 / 부족은 ...입니다"),
        ("One of the major drawbacks to adopting / using / exploiting this system is …", "이 시스템을 채택 / 사용 / 활용하는 주요 단점 중 하나는 ...입니다"),
        ("This is something of a pitfall / disadvantage …", "이것은 ...의 함정 / 단점입니다"),
        ("A well-known / major / serious criticism of X is …", "X에 대한 잘 알려진 / 주요 / 심각한 비판은 ...입니다")
    ],
    "Highlighting limitations of previous studies - authors mentioned by name": [
        ("Peng [31] claimed / contended that X is … but she failed to provide adequate proof of this finding.", "Peng [31]은 X가 ...이라고 주장했지만, 그녀는 이 발견에 대한 적절한 증거를 제공하지 못했습니다"),
        ("Peng’s findings do not seem / appear to support his conclusions.", "Peng의 발견은 그의 결론을 지지하지 않는 것 같습니다"),
        ("This has led authors such as / for example / for instance Mithran [32], Yasmin [34] and Hai [35] to investigate ..", "이로 인해 Mithran [32], Yasmin [34] 및 Hai [35]와 같은 저자들이 ...을 조사하게 되었습니다"),
        ("The shortcomings / pitfalls / flaws of their method have been clearly recognized.", "그들의 방법의 단점 / 함정 / 결함은 분명히 인식되었습니다"),
        ("A serious weakness / limitation / drawback with this argument, however, is that …", "그러나 이 주장에는 심각한 약점 / 제한 / 결함이 있습니다")
    ],
    "Using the opinions of others to justify your criticism of someone’s work": [
        ("As mentioned by Burgess [2011], Henri’s argument / approach / reasoning relies too heavily on …", "Burgess [2011]이 언급한 것처럼, Henri의 주장 / 접근 / 추론은 ...에 너무 의존합니다"),
        ("As others have highlighted [34, 45, 60], Ozil’s approach raises many doubts / is questionable …", "다른 사람들이 강조한 것처럼 [34, 45, 60], Ozil의 접근 방식은 많은 의문을 제기합니다 / 의심스럽습니다"),
        ("Several authors / experts / researchers / analysts have expressed doubts about / called into question / challenged Guyot on the grounds that …..", "여러 저자 / 전문가 / 연구자 / 분석가들이 ...의 이유로 Guyot에 대해 의문을 제기하고 도전했습니다"),
        ("Marchesi [2010] has already noted an inconsistency with Hahn’s claim / methodology / method / results / approach …", "Marchesi [2010]은 Hahn의 주장 / 방법론 / 방법 / 결과 / 접근 방식의 불일치를 이미 지적했습니다"),
        ("Friedrich’s approach [2013] has not escaped criticism / been subjected to much criticism and has been strongly / vigorously challenged …", "Friedrich의 접근 방식 [2013]은 비판을 피하지 못했으며 많은 비판을 받았으며 강력하게 도전받았습니다")
    ],
    "Describing purpose of testing / methods used": [
        ("In order to identify / understand / investigate / study / analyze X …", "X를 식별 / 이해 / 조사 / 연구 / 분석하기 위해 ..."),
        ("To enable / allow us to … , we …", "우리가 ...할 수 있도록, 우리는 ..."),
        ("To see / determine / check / verify / determine whether …", "...인지 확인 / 결정 / 확인 / 검증하기 위해"),
        ("To control / test for X, Y was done.", "X를 제어 / 테스트하기 위해 Y가 수행되었습니다"),
        ("So that we could / would be able to do X, we …", "우리가 X를 할 수 있도록, 우리는 ...")
    ],
    "Outlining similarities with other authors’ models, systems etc.": [
        ("The set up we used can be found / is reported / is detailed in [Ref 2].", "우리가 사용한 설정은 [참조 2]에 나와 있습니다 / 보고되었습니다 / 자세히 설명되어 있습니다"),
        ("Our experimental set up bears a close resemblance to / is reminiscent of / is based on / is a variation on / was inspired by / owes a lot to / is more or less identical to / is practically the same as the one proposed by Smith [2014].", "우리의 실험 설정은 Smith [2014]이 제안한 것과 거의 동일합니다 / 영감을 받았습니다 / 기반으로 합니다 / 변형입니다 / 유사합니다 / 실질적으로 동일합니다"),
        ("We used a variation of Smith’s procedure. In fact / Specifically, in our procedure we …", "우리는 Smith의 절차의 변형을 사용했습니다. 사실 / 구체적으로, 우리의 절차에서 우리는 ..."),
        ("Our steps proceed very much in the same way as / follow what is indicated in [Ref. 2]. First, …", "우리의 단계는 [참조 2]에 나와 있는 것과 매우 유사하게 진행됩니다. 먼저, ..."),
        ("The procedure used is as described / explained / reported / proposed by Sakamoto [2013].", "사용된 절차는 Sakamoto [2013]에 의해 설명 / 설명 / 보고 / 제안된 것입니다")
    ],
    "Describing the apparatus and materials used and their source": [
        ("The instrument used / utilized / adopted / employed was …", "사용된 / 채택된 / 적용된 기기는 ..."),
        ("The apparatus consists of / is made up of / is composed of / is based on …", "기기는 ...로 구성되어 있습니다 / ...에 기반합니다"),
        ("The device was designed / developed / set up in order to …", "기기는 ...하기 위해 설계 / 개발 / 설정되었습니다"),
        ("X incorporates / exploits / makes use of the latest technological advances.", "X는 최신 기술 진보를 통합 / 활용합니다"),
        ("The system comes complete / is equipped / is fully integrated / is fitted with a …", "시스템은 ...로 완전히 통합되어 있습니다 / 장착되어 있습니다")
    ],
    "Reporting software used": [
        ("The software application / program / package used to analyze the data was SoftGather (Softsift plc, London).", "데이터를 분석하는 데 사용된 소프트웨어 응용 프로그램 / 프로그램 / 패키지는 SoftGather (Softsift plc, London)였습니다"),
        ("The data were obtained / collected using SoftGather.", "데이터는 SoftGather를 사용하여 수집되었습니다"),
        ("Data management / analysis was performed by / using SoftGather.", "데이터 관리는 SoftGather를 사용하여 수행되었습니다"),
        ("X was carried out / performed / analyzed / calculated / determined using SoftGather.", "X는 SoftGather를 사용하여 수행되었습니다 / 분석되었습니다 / 계산되었습니다 / 결정되었습니다"),
        ("Statistical significance was analyzed by using / through the use of SoftGather.", "통계적 유의성은 SoftGather를 사용하여 분석되었습니다")
    ],
    "Reporting customizations performed": [
        ("X was tailored / customized for use with …", "X는 ...에 사용되도록 맞춤 제작되었습니다"),
        ("X can easily be customized / adapted / modified to suit all requirements.", "X는 모든 요구 사항에 맞게 쉽게 맞춤 제작 / 조정 / 수정될 수 있습니다"),
        ("Measurements were taken using purpose-built / custom-built / customized equipment.", "측정은 목적에 맞게 제작된 / 맞춤 제작된 장비를 사용하여 수행되었습니다"),
        ("The apparatus was adapted as in [Ref] / in accordance with [Ref] / as follows:", "기기는 [참조]에 따라 / 다음과 같이 조정되었습니다"),
        ("The following changes / modifications were made:", "다음과 같은 변경 / 수정이 이루어졌습니다")
    ],
    "Formulating equations, theories and theorems": [
        ("This problem can be outlined / phrased / posed in terms of …", "이 문제는 ...의 측면에서 개요화 / 표현 / 제시될 수 있습니다"),
        ("The problem is ruled by / governed by / related to / correlated to …", "문제는 ...에 의해 지배됩니다 / 관련됩니다"),
        ("This theorem asserts / states that …", "이 정리는 ...을 주장합니다 / 진술합니다"),
        ("The resulting integrals / solution to X can be expressed as …", "결과적인 적분 / X에 대한 해결책은 ...로 표현될 수 있습니다"),
        ("… where T stands for / denotes / identifies / is an abbreviation for time.", "… 여기서 T는 시간을 나타냅니다 / 의미합니다 / 식별합니다")
    ],
    "Explaining why you chose your specific method, model, equipment, sample etc.": [
        ("The aim / purpose of X is to do Y. Consequently we / As a result we / Therefore we / We thus …", "X의 목적 / 목표는 Y를 하는 것입니다. 따라서 우리는 ..."),
        ("This method / model / system was chosen because it is one of the most practical / feasible / economic / rapid ways to …", "이 방법 / 모델 / 시스템은 ...하는 가장 실용적이고 / 실현 가능하고 / 경제적이고 / 신속한 방법 중 하나이기 때문에 선택되었습니다"),
        ("We chose this particular apparatus because / on account of the fact that / due to / since …", "우리는 이 특정 기기를 ... 때문에 선택했습니다"),
        ("It was decided that the best procedure / method / equipment for this investigation / study was to …", "이 조사 / 연구를 위한 최상의 절차 / 방법 / 장비는 ...하기로 결정되었습니다"),
        ("An X approach was chosen / selected in order to …", "X 접근 방식이 ...하기 위해 선택되었습니다")
    ],
    "Explaining the preparation of samples, solutions etc.": [
        ("We used reliable / innovative / classic / traditional techniques based on the recommendations of …", "우리는 ...의 권장 사항을 기반으로 신뢰할 수 있는 / 혁신적인 / 고전적인 / 전통적인 기술을 사용했습니다"),
        ("Xs were prepared as described by / according to / following Jude [2010].", "Xs는 Jude [2010]에 의해 설명된 대로 / ...에 따라 준비되었습니다"),
        ("Xs were prepared in accordance with / in compliance with / as required by….", "Xs는 ...에 따라 / 준수하여 준비되었습니다"),
        ("Y was prepared using the same / a similar procedure as for X.", "Y는 X와 동일한 / 유사한 절차를 사용하여 준비되었습니다"),
        ("All samples were carefully / thoroughly checked for …", "모든 샘플은 ...에 대해 신중하게 / 철저히 검사되었습니다")
    ],
    "Outlining selection procedure for samples, surveys etc.": [
        ("The traditional / classical / normal / usual approach to sample collection is to …", "샘플 수집에 대한 전통적인 / 고전적인 / 일반적인 / 일반적인 접근 방식은 ...입니다"),
        ("The criteria / reasons for selecting Xs were:", "Xs를 선택한 기준 / 이유는 다음과 같습니다"),
        ("The sample was selected / subdivided on the basis of X and Y.", "샘플은 X와 Y를 기준으로 선택 / 세분화되었습니다"),
        ("The initial sample consisted of / was made up / was composed of …", "초기 샘플은 ...로 구성되었습니다"),
        ("Approximately / Just over / Slightly under a half / third / quarter of the sample were …", "대략 / 절반 이상 / 약간 적은 절반 / 삼분의 일 / 사분의 일의 샘플은 ...였습니다")
    ],
    "Indicating the time frame (past tenses)": [
        ("Initial studies were made / performed / done / carried out / executed using the conditions described above over / for a period of …", "초기 연구는 위에서 설명한 조건을 사용하여 ... 기간 동안 수행되었습니다"),
        ("X was collected / used / tested / characterized / assessed during the first / initial step.", "X는 첫 번째 / 초기 단계 동안 수집 / 사용 / 테스트 / 특성화 / 평가되었습니다"),
        ("Prior to / Before doing X, we did Y.", "X를 하기 전에 우리는 Y를 했습니다"),
        ("First we estimated / determined the value of X, then / subsequently we studied / analyzed / evaluated Y.", "먼저 우리는 X의 가치를 추정 / 결정하고, 그 다음에 / 이어서 Y를 연구 / 분석 / 평가했습니다"),
        ("Once / As soon as / After X had been done, we then did Y.", "X가 완료되자마자 / 완료된 후 우리는 Y를 했습니다")
    ],
    "Indicating the time frame in a general process (present tenses)": [
        ("In the first step / During the first phase / In the initial stage of the process …", "첫 번째 단계에서 / 첫 번째 단계 동안 / 과정의 초기 단계에서 ..."),
        ("Once / As soon as / After X has been done, we can then do Y.", "X가 완료되면 / 완료되자마자 우리는 Y를 할 수 있습니다"),
        ("This sets the stage / We are now ready for the next step.", "이것은 단계를 설정합니다 / 우리는 이제 다음 단계를 준비합니다"),
        ("At this point / Now X can be …", "이 시점에서 / 이제 X는 ..."),
        ("After / When / As soon as these steps have been carried out, X …", "이 단계가 수행된 후 / 수행되자마자 X ...")
    ],
    "Indicating that care must be taken": [
        ("To do this entails / involves / requires doing X.", "이 작업을 수행하려면 X를 수행해야 합니다"),
        ("It is seldom / rarely / usually / generally / often / always practical to …", "...하는 것은 거의 / 거의 / 보통 / 일반적으로 / 종종 / 항상 실용적입니다"),
        ("Considerable / Great care must be taken / exercised when …", "...할 때 상당한 / 많은 주의를 기울여야 합니다"),
        ("A great deal of / Considerable attention must be paid when …", "...할 때 많은 / 상당한 주의를 기울여야 합니다"),
        ("Extreme caution must be taken / used when …", "...할 때 극도의 주의를 기울여야 합니다")
    ],
    "Describing benefits of your method, equipment etc.": [
        ("This method represents a viable / valuable / useful / groundbreaking / innovative alternative to …", "이 방법은 ...에 대한 실행 가능한 / 가치 있는 / 유용한 / 획기적인 / 혁신적인 대안을 나타냅니다"),
        ("This equipment has the ability / capacity / potential to outperform all previous Xs.", "이 장비는 이전의 모든 X를 능가할 수 있는 능력 / 용량 / 잠재력을 가지고 있습니다"),
        ("This apparatus has several / many interesting features / characteristics.", "이 기기는 여러 가지 / 많은 흥미로운 기능 / 특성을 가지고 있습니다"),
        ("Our method has many interesting / attractive / beneficial / useful / practical / effective / valuable applications.", "우리의 방법은 많은 흥미롭고 / 매력적이고 / 유익하고 / 유용하고 / 실용적이고 / 효과적이고 / 가치 있는 응용 프로그램을 가지고 있습니다"),
        ("Of particular / major / fundamental interest is …", "특히 / 주요 / 근본적인 관심사는 ...입니다")
    ],
    "Outlining alternative approaches": [
        ("A less lengthy / time-consuming / cumbersome / costly approach is …", "덜 길고 / 시간이 많이 걸리고 / 번거롭고 / 비용이 많이 드는 접근 방식은 ...입니다"),
        ("A neater / more elegant / simplified / more practical solution for this problem …", "이 문제에 대한 더 깔끔하고 / 더 우아하고 / 간소화되고 / 더 실용적인 솔루션은 ...입니다"),
        ("An alternative solution, though with high overheads / slightly more complicated / less exhaustive is …", "높은 오버헤드 / 약간 더 복잡하고 / 덜 철저한 대안 솔루션은 ...입니다"),
        ("One / One possible / A good way to avoid the use of X is to use Y instead.", "X의 사용을 피하는 한 가지 방법 / 가능한 방법 / 좋은 방법은 대신 Y를 사용하는 것입니다")
    ],
    "Explaining how you got your results": [
        ("To assess X / evaluate X / distinguish between X and Y, Z was used.", "X를 평가하기 위해 / X와 Y를 구별하기 위해 Z가 사용되었습니다"),
        ("X analysis was used to test / predict / confirm Y.", "Y를 테스트 / 예측 / 확인하기 위해 X 분석이 사용되었습니다"),
        ("Changes in X were identified / calculated / compared using …", "X의 변화는 ...를 사용하여 식별 / 계산 / 비교되었습니다"),
        ("The correlation / difference between X and Y was tested.", "X와 Y 사이의 상관 관계 / 차이가 테스트되었습니다"),
        ("The first set of analyses investigated / examined / confirmed / highlighted the impact of …", "첫 번째 분석 세트는 ...의 영향을 조사 / 검토 / 확인 / 강조했습니다")
    ],
    "Reporting results from questionnaires and interviews": [
        ("Of the study population / initial sample / initial cohort, 90 subjects completed and returned the questionnaire.", "연구 인구 / 초기 샘플 / 초기 코호트 중 90명이 설문지를 작성하여 반환했습니다"),
        ("The response rate was 70% at / after / for the first six months and …", "응답률은 처음 6개월 동안 / 후에 / 70%였습니다"),
        ("The majority of respondents / those who responded felt that …", "응답자의 대부분 / 응답한 사람들은 ...라고 느꼈습니다"),
        ("Over half / Sixty per cent of those surveyed / questioned reported that …", "조사 / 질문받은 사람들의 절반 이상 / 60%는 ...라고 보고했습니다"),
        ("Almost / Just under / Approximately two-thirds of the participants (64%) said / felt / commented that …", "참가자의 거의 / 약간 적은 / 약 3분의 2 (64%)는 ...라고 말했습니다 / 느꼈습니다 / 언급했습니다")
    ],
    "Stating what you found": [
        ("These tests revealed / showed / highlighted that …", "이 테스트는 ...을 드러냈습니다 / 보여주었습니다 / 강조했습니다"),
        ("Strong / Some / No evidence of X was found …", "X의 강력한 / 일부 / 증거가 없습니다"),
        ("Interestingly / Surprisingly / Unexpectedly, for high values of X, Y was found ..", "흥미롭게도 / 놀랍게도 / 예기치 않게 X의 높은 값에서 Y가 발견되었습니다"),
        ("There was a significant positive / no correlation between …", "... 사이에 유의한 양의 상관 관계 / 상관 관계가 없었습니다"),
        ("On average / Generally speaking / Broadly speaking, we found values for X of …", "평균적으로 / 일반적으로 / 넓게 말하면, 우리는 X의 값을 ...로 발견했습니다")
    ],
    "Stating what you did not find": [
        ("No significant difference / correlation was found / identified / revealed / detected / observed / highlighted between …. ", "... 사이에 유의한 차이 / 상관 관계가 발견되지 않았습니다 / 식별되지 않았습니다 / 드러나지 않았습니다 / 감지되지 않았습니다 / 관찰되지 않았습니다 / 강조되지 않았습니다"),
        ("There were no significant differences between X and Y in terms of Z / with regard to Z / as far as Z is concerned.", "Z와 관련하여 / Z와 관련하여 / Z와 관련하여 X와 Y 사이에 유의한 차이가 없었습니다"),
        ("The analysis did not show / reveal / identify / confirm any significant differences between …", "분석은 ... 사이의 유의한 차이를 보여주지 않았습니다 / 드러나지 않았습니다 / 식별하지 않았습니다 / 확인하지 않았습니다"),
        ("None of these differences were / Not one of these differences was statistically significant.", "이 차이 중 어느 것도 / 어느 하나도 통계적으로 유의하지 않았습니다"),
        ("Overall / Taken as a whole / Generally speaking / With a few exceptions, our results show X did not affect Y.", "전반적으로 / 전체적으로 / 일반적으로 / 몇 가지 예외를 제외하고, 우리의 결과는 X가 Y에 영향을 미치지 않았음을 보여줍니다")
    ],
    "Highlighting significant results and achievements": [
        ("The most striking / remarkable result to emerge from the data is that …", "데이터에서 나타난 가장 눈에 띄는 / 주목할만한 결과는 ...입니다"),
        ("Interestingly / Curiously / Remarkably / Inexplicably, this correlation is related to …. ", "흥미롭게도 / 호기심에서 / 주목할만하게 / 설명할 수 없게도, 이 상관 관계는 ...와 관련이 있습니다"),
        ("Significantly / Importantly / Crucially / Critically, X is …", "유의하게 / 중요하게 / 결정적으로 / 비판적으로 X는 ...입니다"),
        ("The correlation between X and Y is interesting / of interest / worth noting / noteworthy / worth mentioning because …", "X와 Y 사이의 상관 관계는 ... 때문에 흥미롭습니다 / 흥미롭습니다 / 주목할만한 가치가 있습니다 / 주목할만한 가치가 있습니다 / 언급할만한 가치가 있습니다"),
        ("The most surprising / remarkable / intriguing correlation is with the …", "가장 놀라운 / 주목할만한 / 흥미로운 상관 관계는 ...와 관련이 있습니다")
    ],
    "Stating that your results confirm previous evidence": [
        ("Our experiments confirm / corroborate / are in line with / are consistent with previous results [Wiley 2009].", "우리의 실험은 이전 결과 [Wiley 2009]를 확인합니다 / 확증합니다 / 일치합니다 / 일관됩니다"),
        ("The values are barely / scarcely / hardly distinguishable from [Li 2010] who …", "값은 [Li 2010]과 거의 구별할 수 없습니다"),
        ("This value has been found to be / is typical of X.", "이 값은 X의 전형으로 밝혀졌습니다 / 전형입니다"),
        ("This is in good agreement / in complete agreement / consistent with …", "이것은 ...와 잘 일치합니다 / 완전히 일치합니다 / 일치합니다"),
        ("This fits / matches / concurs well with [65] and also confirms our earlier / previous findings [39, 40, 41].", "이것은 [65]와 잘 맞으며 우리의 이전 / 이전 발견 [39, 40, 41]을 확인합니다")
    ],
    "Stating that your results are in contrast with previous evidence": [
        ("It was found that X = 2, whereas / on the other hand Kamatchi [2011] found that …", "X = 2임이 발견되었지만, 반면에 Kamatchi [2011]은 ...을 발견했습니다"),
        ("We found much higher values for X than / with respect to those reported by Pandey [2000].", "우리는 Pandey [2000]에 의해 보고된 것보다 X에 대해 훨씬 높은 값을 발견했습니다"),
        ("Although / Despite the fact that Li and Mithran [2014] found that X = 2 we found that X = 3.", "Li와 Mithran [2014]이 X = 2라고 발견했음에도 불구하고 우리는 X = 3이라고 발견했습니다"),
        ("In contrast to / contradiction with earlier findings [Castenas, 2009], we …", "이전 연구 [Castenas, 2009]과 대조적으로, 우리는 ..."),
        ("This study has not confirmed previous research on X. However / Nevertheless / Despite this, it serves to …", "이 연구는 X에 대한 이전 연구를 확인하지 않았습니다. 그러나 / 그럼에도 불구하고 / 그럼에도 불구하고, 그것은 ...에 기여합니다")
    ],
    "Stating and justifying the acceptability of your results": [
        ("As expected / anticipated / predicted / forecast / hypothesized, our experiments show / demonstrate / prove that …", "예상대로 / 예측대로 / 예측대로 / 가설대로, 우리의 실험은 ...을 보여줍니다 / 증명합니다"),
        ("Our formula captures / reproduces the response of …", "우리의 공식은 ...의 응답을 캡처 / 재현합니다"),
        ("We found a very good agreement between X and Y.", "우리는 X와 Y 사이에 매우 좋은 일치를 발견했습니다"),
        ("There is an excellent correlation between …", "... 사이에 훌륭한 상관 관계가 있습니다"),
        ("The close correlation between X and Y indicates / suggests / reveals that …", "X와 Y 사이의 밀접한 상관 관계는 ...을 나타냅니다 / 시사합니다 / 드러냅니다")
    ],
    "Justifying your results when previous results are not available or difficult to obtain": [
        ("No results for X have been reported previously. However, …", "X에 대한 결과는 이전에 보고된 적이 없습니다. 그러나, ..."),
        ("These are the first results reported / published on X.", "이것은 X에 대해 보고된 / 발표된 첫 번째 결과입니다"),
        ("Owing to the lack of data on X, we can only compare our results to Y.", "X에 대한 데이터 부족으로 인해 우리는 우리의 결과를 Y와만 비교할 수 있습니다"),
        ("To the best of our knowledge, these results are unprecedented / have not been described before.", "우리가 아는 한, 이 결과는 전례가 없으며 / 이전에 설명되지 않았습니다"),
        ("Although there are no / very few / hardly any studies to compare these results with, they seem to be in line with …", "이 결과와 비교할 연구가 없거나 / 거의 없거나 / 거의 없지만, 그들은 ...와 일치하는 것 같습니다")
    ],
    "Evaluating your findings": [
        ("This finding / result / outcome is likely to be / may be / could be related to …", "이 발견 / 결과 / 결과는 ...와 관련이 있을 수 있습니다 / 관련될 수 있습니다"),
        ("The possible / probable / likely reason for this is that …", "이것에 대한 가능한 / 가능한 / 가능한 이유는 ...입니다"),
        ("The reason for this is not clear / immediately apparent / immediately obvious.", "이것에 대한 이유는 분명하지 않습니다 / 즉시 명백하지 않습니다 / 즉시 명백하지 않습니다"),
        ("We were unable / unable to find any correlation between X and Y.", "우리는 X와 Y 사이의 상관 관계를 찾을 수 없었습니다 / 찾을 수 없었습니다"),
        ("An alternative / another possible explanation for this might be …", "이것에 대한 대안 / 또 다른 가능한 설명은 ...일 수 있습니다")
    ],
    "Indicating confidence / lack of confidence in your findings": [
        ("We are confident / fairly certain / certain / convinced that our results can be extrapolated / transferred to …", "우리는 우리의 결과가 ...에 외삽 / 전송될 수 있다고 확신합니다 / 꽤 확신합니다 / 확신합니다 / 확신합니다"),
        ("This is a promising / encouraging finding, which is / is not in line with our expectations.", "이것은 우리의 기대와 일치하는 / 일치하지 않는 유망한 / 고무적인 발견입니다"),
        ("However, we are less confident about …", "그러나, 우리는 ...에 대해 덜 확신합니다"),
        ("Despite the promising results, this finding needs to be confirmed / validated / verified by a larger sample size.", "유망한 결과에도 불구하고, 이 발견은 더 큰 샘플 크기에 의해 확인 / 검증 / 검증될 필요가 있습니다"),
        ("It is unclear / debatable / questionable / uncertain whether these findings can be applied to …", "이 발견이 ...에 적용될 수 있는지는 불분명합니다 / 논란의 여지가 있습니다 / 의문입니다 / 불확실합니다")
    ],
    "Mentioning problems that were encountered": [
        ("One problem / issue / difficulty we encountered was …", "우리가 직면한 한 가지 문제 / 문제 / 어려움은 ...입니다"),
        ("We encountered / experienced / faced a number of problems with …", "우리는 ...에 많은 문제를 만났습니다 / 경험했습니다 / 직면했습니다"),
        ("There were several limitations / constraints / difficulties in …", "여러 제한 사항 / 제약 / 어려움이 있었습니다"),
        ("Another problem was that / with …", "또 다른 문제는 ...이었습니다"),
        ("It is important to note / recognize / be aware that …", "중요한 것은 ...를 주목 / 인식 / 인식하는 것입니다")
    ],
    "Explaining an unexpected result": [
        ("The unexpected result / finding was that …", "예기치 않은 결과 / 발견은 ...입니다"),
        ("Interestingly / Surprisingly / Unexpectedly, we found that …", "흥미롭게도 / 놀랍게도 / 예기치 않게 우리는 ...을 발견했습니다"),
        ("One unexpected result was …", "한 가지 예기치 않은 결과는 ...이었습니다"),
        ("It is unclear / unclear why …", "왜 ...인지 불분명합니다 / 불명확합니다"),
        ("Possible explanations for this include …", "이에 대한 가능한 설명은 ...을 포함합니다")
    ],
    "Proposing future work": [
        ("Further work / research / investigation is needed to …", "추가 작업 / 연구 / 조사가 필요합니다 ..."),
        ("Future studies should focus on …", "미래의 연구는 ...에 초점을 맞춰야 합니다"),
        ("Further work is required to determine / confirm / establish …", "추가 작업은 ...을 결정 / 확인 / 설정하기 위해 필요합니다"),
        ("Future research could explore / examine …", "미래의 연구는 ...을 탐구 / 검토할 수 있습니다"),
        ("It would be interesting / valuable to investigate …", "...을 조사하는 것은 흥미로울 / 가치 있을 것입니다")
    ],
    "Concluding your paper": [
        ("In conclusion, we have demonstrated that …", "결론적으로, 우리는 ...을 입증했습니다"),
        ("To sum up, our findings suggest that …", "요약하면, 우리의 발견은 ...을 시사합니다"),
        ("Overall, our results indicate that …", "전반적으로, 우리의 결과는 ...을 나타냅니다"),
        ("In summary, we have shown that …", "요약하면, 우리는 ...을 보여주었습니다"),
        ("Taken together, our findings support the hypothesis that …", "종합적으로, 우리의 발견은 ...라는 가설을 지지합니다")
    ]
}

import gradio as gr
import pandas as pd

def get_phrases(category):
    data = categories.get(category, [])
    df = pd.DataFrame(data, columns=["English", "Korean"])
    return df

def display_phrases(category):
    df = get_phrases(category)
    return gr.DataFrame(df)

category_list = list(categories.keys())
interface = gr.Interface(fn=display_phrases, inputs=gr.Dropdown(category_list), outputs="dataframe", live=True)

interface.launch()


def get_phrases(category):
    data = categories.get(category, [])
    df = pd.DataFrame(data, columns=["English", "Korean"])
    return df

# Create the Gradio interface
def display_phrases(category):
    df = get_phrases(category)
    return gr.DataFrame(df)

category_list = list(categories.keys())
interface = gr.Interface(fn=display_phrases, inputs=gr.Dropdown(category_list), outputs="dataframe", live=True)

interface.launch()


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://4d351a30887dd0eebd.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://4c6a936e676bd49796.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
